In [ ]:
import pandas as pd
import numpy as np
import pickle

In [ ]:
image_feat = np.loadtxt('./../SimCLR/cmmd_image_feature.csv',delimiter=',',dtype=np.float32)
ids = pd.read_csv('./../SimCLR/cmmd_ids.csv',header=None)

In [ ]:
ids_list = []
for i in range(len(image_feat)):
    a = 'D' + str(ids[0][i])[0] +'-' + str(ids[0][i])[1:5]
    ids_list.append(a)
    i+=1

In [ ]:
data = pd.read_excel('./CMMD_clinicaldata_revision (1).xlsx')


In [ ]:
data.drop(columns='number', inplace=True)

In [ ]:
data['subtype'].fillna(0,inplace=True)

In [ ]:
data['age_'] = pd.qcut(data['Age'],5,labels=False)

In [ ]:
data.drop(columns=['Age'],inplace=True)

In [ ]:
data['label'] = data['classification'].apply(lambda x: 0 if x == 'Benign' else 1)

In [ ]:
data.drop(columns=['classification'],inplace=True)

In [ ]:
data

In [ ]:
sub = data['ID1']
cdr = data['label']
data_non_dmgi_drop = data.drop(columns=['ID1','label'])
data_dum = pd.get_dummies(data_non_dmgi_drop, columns=data_non_dmgi_drop.keys())

In [ ]:
feature_dict = {}
label_dict = {}
i = 0
# use_clinical_dummy_no = use_clinical_dummy.drop(columns=['cdr','age_','apoe']).fillna(0)

for s in sub:
    feature_dict[s] = data_dum.iloc[i].to_numpy()
    label_dict[s] = cdr[i]
    i+=1

In [ ]:
oasis3_1 = {'label': label_dict, 'feature': feature_dict}
with open('./cmmd.pkl', 'wb') as f:
    pickle.dump(oasis3_1, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
concat_feature = []
labels = []
i =0
for idd in ids_list:
    pp = image_feat[i].tolist() + feature_dict[idd].tolist()
    concat_feature.append(pp)
    labels.append(label_dict[idd])
    i+=1

In [ ]:
concat_num = np.array(concat_feature)

In [ ]:
data

In [ ]:
type_list = [['LeftRight'],['abnormality'],['subtype'],['age_']]
from sklearn.preprocessing import minmax_scale
import sklearn.metrics.pairwise

k = 0
save_list= []
threses = [0.9, 0.9, 0.9, 0.5]
for types in type_list:
    before_adj_ = []
    use_clinical_dummy_multi = pd.get_dummies(data[['label']+types], columns=types)
    ll = use_clinical_dummy_multi.drop(columns=['label'])
    ll = ll.fillna(0)
    for id_ in ids_list:
        # print(data[data['ID1'] ==id_].index[0])
        lp = data[data['ID1'] ==id_].index[0]
        p = ll.loc[lp].tolist()
        before_adj_.append(p)
    p_ = np.array(before_adj_)
    p_ = minmax_scale(p_, axis=0, copy =True)

    cos_ = sklearn.metrics.pairwise.cosine_similarity(p_,p_)
    adj_ = np.zeros(cos_.shape)
    thres = threses[k]
    for i in range(cos_.shape[0]):
        for j in range(cos_.shape[0]):
            if cos_[i][j] > thres:
                adj_[i][j] = 1
            else:
                adj_[i][j] = 0
    save_list.append(adj_)
    k+=1

In [ ]:
adj_0 = save_list[0]
adj_1 = save_list[1]
adj_2 = save_list[2]
adj_3 = save_list[3]

In [ ]:
indexes = [i for i in range(len(labels))]
train_ = len(indexes[:int(len(indexes)*0.6)])
valid_ = len(indexes[int(len(indexes)*0.6):int(len(indexes)*0.7)])
test_ = len(indexes[int(len(indexes)*0.7):])

import random
random.shuffle(indexes)
train_index = indexes[:train_]
valid_index = indexes[train_:train_+valid_]
test_index = indexes[train_+valid_:]


In [ ]:
y = np.zeros((len(labels),2))
for i in range(len(labels)):
    if labels[i] ==0:
        y[i,0]=1
    elif labels[i]==1:
        y[i,1]=1

train_index = np.array(train_index)
valid_index = np.array(valid_index)
test_index = np.array(test_index)

In [ ]:
concat_num.shape

In [ ]:
oasis3_multi = {'label':y,'type0':adj_0,'type1':adj_1,'type2':adj_2,'type3':adj_3,'train_idx':train_index,'val_idx':valid_index,'test_idx':test_index,'feature':concat_num}


In [ ]:
with open('./../../MultiplexNetwork/data/cmmd.pkl', 'wb') as f:
    pickle.dump(oasis3_multi, f, pickle.HIGHEST_PROTOCOL)